In [ ]:
import pandas as pd
import torch
from transformers import pipeline


## Test HuggingFace call


In [ ]:
# sentiment_test = pipeline(
#    "sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

In [ ]:
# response = sentiment_test(
#    "I think my Spotify AI dj is learning that I'm autistic and only like listening to the songs I already have on my playlist"
# )

In [ ]:
# response

## Sentiment Analysis on collected tweets


In [ ]:
tweets = pd.read_csv("data/spotify_ai_dj_tweets_clean.csv")
tweets.head()

,id,text,author_id,created_at,likes,retweets,replies,text_clean
0,1963699578370220543,This playlist gives Spotify AI DJ💀💀,4913255457,2025-09-04 20:23:57+00:00,0,0,0,This playlist gives Spotify AI DJ💀💀
1,1963692075439849536,The Spotify AI DJ randomly played this today.....,60245577,2025-09-04 19:54:09+00:00,0,0,0,The Spotify AI DJ randomly played this today.....
2,1963646985556103628,@g0dbr34th3d Your playlist is amazing for real...,1495824184638906368,2025-09-04 16:54:58+00:00,0,0,0,@user Your playlist is amazing for real.\n\nWo...
3,1963624836846023034,Makes sense that this is why the Spotify AI DJ...,261881081,2025-09-04 15:26:58+00:00,6,1,1,Makes sense that this is why the Spotify AI DJ...
4,1963616655533093015,I’ve only just discovered Spotify AI Dj..,33827542,2025-09-04 14:54:27+00:00,1,0,0,I’ve only just discovered Spotify AI Dj..


In [ ]:
# Run sentiment analysis on the 'text_clean' column using the classifier and device selection above


# Only consider cuda or cpu, ignore mps


device = "cuda" if torch.cuda.is_available() else "cpu"

if device == "cuda":
    classifier = pipeline(
        "sentiment-analysis",
        model="cardiffnlp/twitter-roberta-base-sentiment-latest",
        device=0,
    )
else:
    classifier = pipeline(
        "sentiment-analysis",
        model="cardiffnlp/twitter-roberta-base-sentiment-latest",
        device=-1,
    )


# Get both sentiment and confidence (score) for each tweet
def get_sentiment_and_confidence(text):
    result = classifier(text)[0]
    return pd.Series({"sentiment": result["label"], "confidence": result["score"]})


tweets[["sentiment", "confidence"]] = tweets["text_clean"].apply(
    get_sentiment_and_confidence
)
tweets.head()


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


,id,text,author_id,created_at,likes,retweets,replies,text_clean,sentiment,confidence
0,1963699578370220543,This playlist gives Spotify AI DJ💀💀,4913255457,2025-09-04 20:23:57+00:00,0,0,0,This playlist gives Spotify AI DJ💀💀,negative,0.661658
1,1963692075439849536,The Spotify AI DJ randomly played this today.....,60245577,2025-09-04 19:54:09+00:00,0,0,0,The Spotify AI DJ randomly played this today.....,positive,0.798091
2,1963646985556103628,@g0dbr34th3d Your playlist is amazing for real...,1495824184638906368,2025-09-04 16:54:58+00:00,0,0,0,@user Your playlist is amazing for real.\n\nWo...,positive,0.961897
3,1963624836846023034,Makes sense that this is why the Spotify AI DJ...,261881081,2025-09-04 15:26:58+00:00,6,1,1,Makes sense that this is why the Spotify AI DJ...,negative,0.917595
4,1963616655533093015,I’ve only just discovered Spotify AI Dj..,33827542,2025-09-04 14:54:27+00:00,1,0,0,I’ve only just discovered Spotify AI Dj..,neutral,0.777509
